# Demo

In this demo, we are going to analyse the result of joining two tables created with Hash distribution on different columns. We will also see how to fix it. 

We will start by creating two new tables with Hash distribution on different columns. 

[Guidance for designing distributed tables using dedicated SQL pool in Azure Synapse Analytics](https://docs.microsoft.com/en-us/azure/synapse-analytics/sql-data-warehouse/sql-data-warehouse-tables-distribute)

## Actual State

Dynamic Management Views (DMVs) allow you to monitor your workload, including investigating query execution in SQL pool. You can check more [here] (https://docs.microsoft.com/en-us/azure/synapse-analytics/sql-data-warehouse/sql-data-warehouse-manage-monitor).

We will use the folowing DMVs to see what is running in the DW and what are the query steps:
- [sys.dm_pdw_exec_requests (Transact-SQL)](https://docs.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-pdw-exec-requests-transact-sql?toc=%2Fazure%2Fsynapse-analytics%2Fsql-data-warehouse%2Ftoc.json&bc=%2Fazure%2Fsynapse-analytics%2Fsql-data-warehouse%2Fbreadcrumb%2Ftoc.json&view=azure-sqldw-latest&preserve-view=true) 
- [sys.dm_pdw_request_steps (Transact-SQL)](https://docs.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-pdw-request-steps-transact-sql?toc=%2Fazure%2Fsynapse-analytics%2Fsql-data-warehouse%2Ftoc.json&bc=%2Fazure%2Fsynapse-analytics%2Fsql-data-warehouse%2Fbreadcrumb%2Ftoc.json&view=azure-sqldw-latest&preserve-view=true)


### **sys.dm_pdw_exec_requests**
Holds information about all requests currently or recently active in Azure Synapse Analytics. It lists one row per request/query..

### **sys.dm_pdw_request_steps**
Holds information about all steps that compose a given request or query in Azure Synapse Analytics. It lists one row per query step.

Run the next cell to check if there's any activity currently running on your database.

In [ ]:
SELECT * FROM sys.dm_pdw_exec_requests WHERE 1=1 AND status = 'Running' and session_id != SESSION_ID()

: 

Now, we will create two new tables with Hash distribution on different columns:  Medallion_new and Trip_New:



In [ ]:
CREATE TABLE [dbo].[Medallion_new] WITH (DISTRIBUTION = HASH (MedallionID), CLUSTERED COLUMNSTORE INDEX)
AS SELECT * FROM [dbo].[Trip]

CREATE TABLE [dbo].[Trip_New] WITH (DISTRIBUTION = HASH (HackneyLicenseID), CLUSTERED COLUMNSTORE INDEX)
AS SELECT * FROM [dbo].[Trip]

: 

The new tables have the same columns, as they were created based on the same table: **Trip**. Now, we will check the **data movement** by  joining them. 

In [24]:
SELECT TOP (1000) * FROM [dbo].[Medallion_new] m INNER JOIN [dbo].[Trip_New] t
ON m.MedallionID = t.MedallionID

(1 row affected)

Total execution time: 00:00:00.191

Table_name,Index_name,Partition_total
[dbo].[Trip_Partitioned],ClusteredIndex_b18d11a02b7e48dda241dd83c29aaec9,13


Run the **sys.dm_pdw_exec_requests** immediately after you run the Join script above to get the request_id of the active join, and then check the **sys.dm_pdw_request_steps** to fetch the resulting **ShuffleMove**. 

In [ ]:
SELECT * FROM sys.dm_pdw_exec_requests WHERE 1=1 AND status = 'Running' and session_id != SESSION_ID()
-- After getting the request_id, replace it below and run:
SELECT * FROM sys.dm_pdw_request_steps WHERE 1=1 AND request_id In ('replace me')

We see an incompatible join - both tales are hasehed on different columns. As the join was performed on the MedallionID column, the **Trip_New** table will be resdistributed by the MedallionID, because when it was created it was distributed on the HackneyLicenseID column. This will cause a **ShuffleMove** operation. You will see this move because data in the **Trip_New** table is being redistributed since it has a different distribution column than the original distributed table. This can be solved by changing the distribution key.


Insteade of executing the join query above, you can simply check its **Execution plan**, which will give you a diagram where you can see the operation steps, the effort made by the engine to compute the steps, the hash value, and other execution properties. In SQL Server Management Studio (SSMS), you simply select the query and click on *Display Estimated Execution Plan*. This is the compiled plan, as produced by the Query Optimizer based on estimations. This is the query plan that is stored in the plan cache, and is an important visual aid for troubleshooting and for specific steps where complex queries are a multistep operation. 

Another important tool is the output of the *[EXPLAIN](https://docs.microsoft.com/en-us/sql/t-sql/queries/explain-transact-sql?view=azure-sqldw-latest)* function:  it returns the **Query plan** for a Microsoft Azure Synapse Analytics SQL statement without running the statement. Use EXPLAIN to preview which operations will require data movement and to view the estimated costs of the query operations. WITH RECOMMENDATIONS applies to Azure Synapse Analytics, but is not supported by serverless SQL pool in Azure Synapse Analytics.

## Issue and Fix

We can apply an easy fix for this demo's purpose, but in a real scenario you need to have business knowledge and possibly involve a Data Analyst to chose the appropriate distribution column. 

To avoid a shuffle Move here, we will create the **Trip_New_K** table. **Alter table** does not yet support changing the distribution in a table in Azure Synapse, so you can also create a copy table of the original table using CTAS, drop the original table and then rename the new one to the original name.

### **Fix**
In our demo, we opted for creating a new table with a new distribution key: **MedallionID**.

In [25]:
CREATE TABLE [dbo].[Trip_New_K] WITH (DISTRIBUTION = HASH (MedallionID),CLUSTERED COLUMNSTORE INDEX)
AS SELECT * FROM [dbo].[Trip]

(170261325 rows affected)

Total execution time: 00:04:34.187

Now, we can see there's no more **data movement** when joining the **Medallion_new** and the **Trip_New_K** tables. 

In [ ]:
SELECT TOP (1000) * FROM [dbo].[Medallion_new] m INNER JOIN [dbo].[Trip_New_K] k
ON m.MedallionID = t.MedallionID